# Stellar Metallicity

In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings

In [3]:
figure_setup()

## [Mg/Fe] vs [Fe/H]

In [4]:
def read_data(simulation: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation,
                 loadonlytype=[4])
    s.add_metal_abundance(of="Fe", to="H")
    s.add_metal_abundance(of="Mg", to="Fe")

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    return (s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
            s.metal_abundance["Mg/Fe"][is_real_star & is_main_obj])

In [14]:
def add_panel_for_galaxy(simulation: str, ax: plt.Axes):
    """
    This method adds a panel with the results for a given simulation.

    Parameters
    ----------
    simulation : str
        The simulation.
    ax : plt.Axes
        The ax to which to add the plot.
    """

    fe_to_h, mg_to_fe = read_data(simulation=simulation)
    _, _, _, im = ax.hist2d(fe_to_h,
                            mg_to_fe + 0.4,
                            cmap="nipy_spectral",
                            bins=200,
                            range=[ax.get_xlim(), ax.get_ylim()],
                            norm=mcolors.LogNorm(vmin=1E0, vmax=1E3))
    ax.plot(ax.get_xlim(),
            [np.nanmedian(mg_to_fe + 0.4),
             np.nanmedian(mg_to_fe + 0.4)],
            lw=.25, color='k')
    ax.plot([np.nanmedian(fe_to_h),
             np.nanmedian(fe_to_h)],
            ax.get_ylim(),
            lw=.25, color='k')
    ax.plot(np.nanmedian(fe_to_h),
            np.nanmedian(mg_to_fe + 0.4),
            marker='o', mfc='k', ms=2, mew=0)
    ax.text(x=0.05,
            y=0.05,
            s=r"$\texttt{" + simulation.upper() + "}$",
            size=6.0, transform=ax.transAxes,
            ha='left', va='bottom',
            )

In [20]:
def plot_abundance_rel_for_sample(simulations: list,
                                  filename: str):
    """
    This method creates a plot of the Mg/Fe abundance as a function of the
    Fe\H abundance for all subhalo stars.

    Parameters
    ----------
    simulations : list
        A list of simulations to plot.
    filename : str
        The name of the output file.
    """

    n_simulations = len(simulations)

    fig = plt.figure(figsize=(7.2, 7.2))
    gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    set_axs_configuration(
        xlim=(-2, 1), ylim=(-0.1, 0.4),
        xticks=[-2, -1, 0], yticks=[-0.1, 0, 0.1, 0.2, 0.3],
        xlabel="[Fe/H]", ylabel="$\mathrm{[Mg/Fe]} + 0.4$",
        axs=axs, n_used=n_simulations)

    for idx, ax in enumerate(axs.flat):
        if idx < n_simulations:
            add_panel_for_galaxy(simulations[idx], ax=ax)
        else:
            ax.axis("off")

#     cbar = fig.colorbar(im, ax=ax, orientation='vertical',
#                         label=r'$N_\mathrm{stars}$',
#                         pad=0)
#     cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

    for ext in ["pdf", "png"]:
        fig.savefig(f"../images/MgFe_vs_FeH/{filename}.{ext}")
    plt.close(fig)

In [22]:
# Create plots for all galaxies in two figures
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
reruns = [f"au{i}_re_l4_s251" for i in settings.reruns]
plot_abundance_rel_for_sample(simulations=originals,
                              filename="originals_s127")
plot_abundance_rel_for_sample(simulations=reruns,
                              filename="reruns_s251")

## [Mg/Fe] vs [Fe/H] by Region

In [23]:
def read_data_with_region(simulation: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.

    Returns
    -------
    tuple
        The properties.
    """
    s = Snapshot(simulation=simulation,
                 loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_reference_to_potential()
    s.add_normalized_potential()
    s.tag_particles_by_region(disc_std_circ=1.0,
                              disc_min_circ=0.4,
                              cold_disc_delta_circ=0.25,
                              bulge_max_specific_energy=-0.75)
    s.add_metal_abundance(of="Fe", to="H")
    s.add_metal_abundance(of="Mg", to="Fe")

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    return (s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
            s.metal_abundance["Mg/Fe"][is_real_star & is_main_obj],
            s.region_tag[is_real_star & is_main_obj])

In [24]:
def plot_abundance_rel_by_region_for_galaxy(simulation: str):
    """
    This method creates a plot of the Mg/Fe abundance as a function of the
    Fe\H abundance for each galactic component.

    Parameters
    ----------
    simulation : str
        The simulation to plot.
    """
    settings = Settings()

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(-2, 1)
        ax.set_ylim(-0.1, 0.4)
        ax.set_xticks([-1, 0, 1])
        ax.set_yticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4])
        ax.set_xlabel("[Fe/H] [dex]")
        ax.set_ylabel("$\mathrm{[Mg/Fe]} + 0.4$ [dex]")
        ax.label_outer()

    fe_to_h, mg_to_fe, region_tag = read_data_with_region(
        simulation=simulation)
    for idx, tag in enumerate(np.unique(region_tag)):
        is_region = (region_tag == tag)
        _, _, _, im = axs[idx].hist2d(
            fe_to_h[is_region],
            mg_to_fe[is_region] + 0.4,
            cmap='nipy_spectral',
            bins=200,
            range=[axs[idx].get_xlim(), axs[idx].get_ylim()],
            norm=mcolors.LogNorm(vmin=1E0, vmax=1E3))

        axs[idx].text(x=0.05,
                      y=0.05,
                      s=settings.component_labels[settings.components[idx]],
                      size=8.0, transform=axs[idx].transAxes,
                      ha='left', va='bottom',
                      )

        axs[idx].plot(axs[idx].get_xlim(),
                      [np.nanmedian(mg_to_fe[is_region] + 0.4),
                       np.nanmedian(mg_to_fe[is_region] + 0.4)],
                      lw=.25, color='k')
        axs[idx].plot([np.nanmedian(fe_to_h[is_region]),
                       np.nanmedian(fe_to_h[is_region])],
                      axs[idx].get_ylim(),
                      lw=.25, color='k')
        axs[idx].plot(np.nanmedian(fe_to_h[is_region]),
                      np.nanmedian(mg_to_fe[is_region] + 0.4),
                     marker='o', mfc='k', ms=2, mew=0)

    cbar = fig.colorbar(im, ax=axs[-1], orientation='vertical',
                        label=r'$N_\mathrm{stars}$',
                        pad=0)
    cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

    axs[0].text(x=axs[0].get_xlim()[0],
                y=axs[0].get_ylim()[1],
                s=r"$\texttt{" + simulation.upper() + "}$",
                size=8.0,
                ha='left', va='bottom',
                )

    for ext in ["pdf", "png"]:
        fig.savefig(f"../images/MgFe_vs_FeH_by_region/{simulation}.{ext}")
    plt.close(fig)

In [25]:
plot_abundance_rel_by_region_for_galaxy(simulation="au1_or_l4_s127")
plot_abundance_rel_by_region_for_galaxy(simulation="au6_or_l4_s127")
plot_abundance_rel_by_region_for_galaxy(simulation="au17_or_l4_s127")